In [1]:
import pandas as pd
import pickle as pkl

In [2]:
X_train = pd.read_parquet("data/balanced/X_train.parquet")
X_calib = pd.read_parquet("data/balanced/X_calib.parquet")
y_train = pd.read_parquet("data/balanced/y_train.parquet")
y_calib = pd.read_parquet("data/balanced/y_calib.parquet")

In [3]:
with open("models_files/serialized/logistic_regression.pkl", "rb") as f:
    model = pkl.load(f)

In [5]:
from sklearn.calibration import CalibratedClassifierCV

In [42]:
calibrated = CalibratedClassifierCV(model, cv="prefit", method="isotonic")

In [43]:
calibrated.fit(X_calib, y_calib)

CalibratedClassifierCV(base_estimator=LogisticRegression(C=0.24363315551442388,
                                                         fit_intercept=False,
                                                         max_iter=500,
                                                         penalty='l1',
                                                         random_state=42,
                                                         solver='saga'),
                       cv='prefit', method='isotonic')

In [34]:
X_test = pd.read_parquet("data/balanced/X_test.parquet")
y_test = pd.read_parquet("data/balanced/y_test.parquet")

In [35]:
from models.utils.metrics import get_metrics

In [59]:
(calibrated.predict_proba(X_test)[:, 1] > .08).mean()

0.2083545814571692

In [64]:
class Model:

    def predict(self, X):
        return (calibrated.predict_proba(X_test)[:, 1] > .08)

In [65]:
get_metrics(Model(), X_test, y_test)

{'accuracy': 0.5184940053165519,
 'f1_score': 0.17858068337467145,
 'precision': 0.10355709164287401,
 'recall': 0.6481257557436517,
 'ROC_AUC': 0.5776156740758814}